# Seminarios de Procesos Gaussianos

### Grupo de procesamiento de la información visual (VIP) 

<div style="text-align: right"> Miguel López Pérez - mlopezp@ugr.es </div>

# GUIÓN IV: PUNTOS INDUCTORES Y FITC

En este guión vamos a estudiar como los puntos inductores nos ayudarán a resolver los problemas asociados a la estimación de parámetros y predicción cuando tenemos muchos datos. Veremos una aproximación basada en la aproximación de la distribución a priori (el GP). En sesiones posteriores estudiaremos otros métodos que se basan en la aproximación de la distribución a posteriori del GP dadas las observaciones. Comenzados importando lo que necesitamos.

In [ ]:
import gpflow
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans2

En primer lugar vamos a definir unas funciones auxiliares que pintarán nuestros modelos. 
  
  - *plot* pintará los datos ruidosos observados junto a nuestro proceso Gaussiano. 
  - *plot_Z* pintará los datos ruidosos observados, el proceso Gaussiano y las entradas inductoras estimadas.

In [ ]:
def plot(m):
    xx = np.linspace(-0.1, 1.1, 100).reshape(100, 1)
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(X, Y, 'kx', mew=2, label='datos ruidosos observados')
    plt.plot(xx, mean, 'C0', lw=2)
    plt.fill_between(xx[:,0],
                     mean[:,0] - 2*np.sqrt(var[:,0]),
                     mean[:,0] + 2*np.sqrt(var[:,0]),
                     color='C0', alpha=0.2)
    plt.xlim(-0.1, 1.1)
    
def plot_Z(m):
    xx = np.linspace(-0.1, 1.1, 100).reshape(100, 1)
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(X, Y, 'kx', mew=2, label='datos ruidosos observados')
    plt.plot(xx, mean, 'C0', lw=2)
    Z = m.feature.Z.read_value()
    plt.plot(Z, np.zeros(len(Z)), '*', mew=2, color = 'red', label='entradas inductoras estimadas')
    plt.fill_between(xx[:,0],
                     mean[:,0] - 2*np.sqrt(var[:,0]),
                     mean[:,0] + 2*np.sqrt(var[:,0]),
                     color='C0', alpha=0.2)
    plt.xlim(-0.1, 1.1)

## Fully Independent Training Conditional (FITC)

Cuando tenemos muchos datos los gpS sufren problemas debido al hecho de tener que invertir una matriz demasiado grande. Para resolver esto, están los llamados modelos Sparse que introducen inducing points (puntos inductores). De este modo, la matriz de covarianza tan grande solo deberá ser evaluada en los inducing points reduciendo su tamaño (y, por tanto, facilitando su inversión). 

La primera aproximación es la FITC, en esta aproximación consideramos que $f$ y $f_*$ son condicionalmente independientes dados los inducing points. Es decir, aproximamos $$p(f, f_*) \sim q(f,f_*) = \int q(f_*\mid u) q(f\mid u) p(u) du$$

Mira la presentación para entender como se realiza el aprendizaje y la inferencia

### Ejemplo

Vamos a ver un ejemplo en 1D sencillo para entender el modelo FITC. Para ello, consideramos nuestra función latente $f$:


$$f(x)= 5\exp\left(-35(x-0.1)^2\right) + 4\exp\left(-40(x-0.9)^2\right)$$


Véase que son la suma de dos funciones gaussianas. Consideramos $N=12$ observaciones ruidosas: (con ruido gaussiano aditivo) $\{(x_i, y_i)\},\quad i=1,\dots,N$ 

$$y_i = f(x_i) + \epsilon, \quad \epsilon_i\sim\mathcal{N}(0,0.01), \qquad i=1,\dots,N$$,

In [ ]:
np.random.seed(200) #Semilla para que sea reproducible
N = 12 #Número de puntos observados
X = np.random.rand(N,1) #Rasgos observados
Y = 5 * np.exp(-35*(X-0.1)**2) + 4* np.exp(-40*(X-0.9)**2) + np.random.randn(N,1)*0.1 #Variable objetivo observada

#Función latente
N = 100
X_real = np.linspace(-0.1, 1.1, 100).reshape(100, 1)
Y_real = 5 * np.exp(-35*(X_real-0.1)**2) + 4* np.exp(-40*(X_real-0.9)**2)

#Pintamos los datos
plt.plot(X, Y, 'kx', mew=2, label='datos ruidosos observados') 
plt.plot(X_real, Y_real, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()

#### GP para regresión

Observa que en un probelam real, tú solo tendrás acceso a los datos ruidosos y obviamente no a la función latente.

En la siguiente celda tenemos cual sería el modelo ajustado usando todos los datos observados. También se muestra la log verosimilitud del modelo. Recordamos que la log verosimilitud del modelo es $ \log p(y\mid x, \theta)$. Cuanto más alto sea más verosímiles son los datos ruidosos observados dados los rasgos y los parámetros estimados. Recordamos también que está es la función objetivo a maximizar cuando hallamos los parámetros del modelo.

Ajustamos un modelo de regresión usando un kernel RBF y usando el optimizador Adam.

In [ ]:
sc = 0.1
var = 10
var_lik = 0.01

k = gpflow.kernels.RBF(1, lengthscales=sc, variance = var)
m = gpflow.models.GPR(X, Y, kern=k)
m.likelihood.variance = var_lik
gpflow.training.AdamOptimizer(0.01).minimize(m, maxiter = 200)

plot(m)
plt.plot(X_real, Y_real, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()
print('La log-verosimilitud del modelo es', m.compute_log_likelihood())

Recuerda lo que significan las franjas azules

#### Aproximación para regresión usando GP: FITC

En la siguiente celda, vamos a aplicar el modelo FITC. Este modelo usa puntos inductores, llamaremos entradas (o localizaciones) inductoras a los $Z$ y salidas inductoras $u = f(z)$ el valor de la función latente en las respectivas entradas inductoras. Esto puede liarte un poco. Con el tiempo para $u$ calcularemos una aproximación de una distribución a posteriori pero de momento nos quedamos en el GP original.

Elegiremos un número de puntos inductores $M$.Podemos alterar el número $M$ de puntos inductores y ver que modelos salen con su respectiva log-verosimilitud. Las entradas inductoras se iniciarán usando un algoritmo kmeans ($M$ clusters) sobre los datos, donde nos quedaremos con los respectivos centroides. Al entrenar el modelo, obtendremos las localizaciones de las entradas inductoras así como los parámetros del modelo. Con todo ello, realizaremos las predicciones (mira la presentación de esta sesión).

In [ ]:
M = 5 #Numero de inducing points
sc = 0.1
var = 10
var_lik = 0.01

Z = kmeans2(X, M, minit='points')[0]
k = gpflow.kernels.RBF(1, lengthscales=sc, variance = var)
m = gpflow.models.GPRFITC(X, Y, kern=k, Z = Z) #modelo FITC
m.likelihood.variance = var_lik
gpflow.training.AdamOptimizer(0.01).minimize(m, maxiter = 200)

plot_Z(m)
plt.plot(Z, -1 + np.zeros(len(Z)), '^', mew=2, color = 'blue', label='entradas inductoras iniciales')
plt.plot(X_real, Y_real, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()
print('La log-verosimilitud del modelo es', m.compute_log_likelihood())

Observa esta gráfica, es muy interesante. Hemos dibujado las entradas inductoras. ¿Por qué no hemos dibujado las salidas?. Observa la parte de la izquierda de la gráfica, ¿qué crees que está pasando?.

En la siguiente celda podemos elegir donde colocar las entradas inductoras y ver el GP resultante usando FITC.

In [ ]:
Z = [[0.01], [0.11], [0.24], [0.9], [1]] # Inducing points
m = gpflow.models.SGPR(X, Y, kern=k, Z = Z)
m.feature.set_trainable(False)
plot_Z(m)
plt.plot(X_real, Y_real, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
print('La log-verosimilitud del modelo es', m.compute_log_likelihood())

### Aplicación: denoising o eliminación de ruido de imágenes

Para ilustrar la necesidad de utilizar aproximaciones cuando tenemos un elevado número de datos,  vamos a elegir el problema de denoising de imágenes. Este es un problema inverso (mal planteado) en el campo de restauración de imágenes. El problema consiste en  recuperar la imagen latente sin ruido de una imagen observada con cierto ruido (gaussiano).

Vamos a generar una imagen sintética en escala de grises. A continuación le añadiremos ruido gaussiano.  

La función latente que usaremos será $f$:

$$f(x_1, x_2) = x_1(1-x_1)\cos(4\pi x_1)\sin(4\pi x_2^2)^2$$

Esta función la observaremos en un grid del cuadrado $[0,1]\times[0,1]$, exactamente haremos un grid con $300\times300=90.000$ puntos equidistribuidos. Al valor resultante le añadiremos ruido gaussiano:

$$\epsilon \sim\mathcal(0, 0.08^2)$$

Nuestro problema sería entonces en conseguir recuperar la función latente $f$ que no tiene ruido:

$$y = f(x_1,x_2) + \epsilon$$

Nótese: que al hacer regresión con los procesos gaussianos queremos modelizar la función latente, no queremos modelizar la variable ruidosa observada. Por tanto, los procesos gaussianos nos valen para hacer denoising.

In [ ]:
#Definimos la función latente y el grid donde la evaluaremos
def func(x, y):
     return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2
grid_x, grid_y = np.mgrid[0:1:300j, 0:1:300j]

#Generamos la imagen A
A = func(grid_x, grid_y)
#Creamos la imagen ruidosa de A
ruido = np.random.normal(0, 0.08, A.shape)
A_ruidosa = A + ruido

Mostramos la imagen latente y la imagen ruidosa observada

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6,6))
ax = axes.ravel()

ax[0].imshow(A, cmap='gray')
ax[0].set_title('A')
ax[0].set_axis_off()

ax[1].imshow(A_ruidosa,
             extent=(0,1,0,1),
             cmap='gray')
ax[1].set_title('A_ruidosa')
plt.tight_layout()
plt.show()

Generamos nuestro dataset, tendremos $2$ rasgos $(x_1, x_2)$ por cada instancia, los rasgos serán la posición espacial, y una variable ruidosa observada $y$. Nuestro dataset tendría la siguiente forma:


|   |   $x_1$  |   $x_2$  |     y    |
|:-:|:--------:|:--------:|:--------:|
|   |     0    |     0    |  0.05585 |
|   |     0    |  0.00334 |  0.00425 |
|   |     0    |  0.00668 |  0.65891 |
|   | $\vdots$ | $\vdots$ | $\vdots$ |

In [ ]:
# Construimos un "dataset" que contendrá las coordenadas de cada punto
from itertools import product
l = np.array(list(product(np.linspace(0,1,300), np.linspace(0,1,300))))

#La variable objetivo serán los valores observados (A ruidosa)
Y = A_ruidosa.reshape(-1,1)

Para este problema no podemos usar los GP ya que el tamaño de la imagen es $300\times 300$ tenemos $90.000$ datos.

Nos devuelve el error: *ResourceExhaustedError: OOM when allocating tensor with shape[90000,90000] *

In [ ]:
sc = 0.1
var = 1
var_lik = 0.08
k = gpflow.kernels.RBF(2, lengthscales=sc, variance = var)
m = gpflow.models.GPR(l, Y, kern=k)

#Optimización del modelo
gpflow.training.AdamOptimizer(0.01).minimize(m, maxiter=100)

Vamos a usar la aproximación FITC con 100 puntos inductores usando dos kernels distintos: RBF y Matern32. Vamos primero a realizar las estimaciones e inferencias correspondientes y luego dibujaremos todos los resultados. Va a tardar un poco, sería bueno que mientras tanto pensases lo que va a salir.

** Kernel: RBF **

In [ ]:
#Modelo con 100 inducing points
M = 100
Z = kmeans2(l, M, minit='points')[0]

sc = 0.1
var = 1
var_lik = 0.08
k = gpflow.kernels.RBF(2, lengthscales=sc, variance = var)
m = gpflow.models.GPRFITC(l, A_ruidosa.reshape(-1,1), kern=k, Z=Z)

#Optimización del modelo
gpflow.training.AdamOptimizer(0.01).minimize(m, maxiter=100)

#Predicción
Y_pred, var = m.predict_y(l)
Y_RBF = Y_pred.reshape(300,300)

** Kernel: Matern32 **

In [ ]:
#Modelo con 100 inducing points
M = 100
Z = kmeans2(l, M, minit='points')[0]

sc = 0.1
var = 1
var_lik = 0.08
k = gpflow.kernels.Matern32(2, lengthscales=sc, variance = var)
m1 = gpflow.models.GPRFITC(l, A_ruidosa.reshape(-1,1), kern=k, Z=Z)

#Optimización del modelo
gpflow.training.AdamOptimizer(0.01).minimize(m1, maxiter=100)

#Predicción
Y_pred, var = m1.predict_y(l)
Y_Matern = Y_pred.reshape(300,300)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8,8))
ax = axes.ravel()

ax[0].imshow(A, cmap='gray',
             extent=(0,1,0,1))
ax[0].set_title('A')
ax[0].set_axis_off()

ax[1].imshow(A_ruidosa,
             extent=(0,1,0,1),
             cmap='gray')
ax[1].set_title('A - ruidosa')

ax[2].imshow(Y_RBF, cmap='gray',
             extent=(0,1,0,1))
ax[2].set_title('A - RBF')

ax[3].imshow(Y_Matern,
             extent=(0,1,0,1),
             cmap='gray')
ax[3].set_title('A - Matern')

plt.tight_layout()
plt.show()

Por último, antes de dibujar todo, vamos a incluir también las posiciones de las entradas inductoras

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8,8))
ax = axes.ravel()

ax[0].imshow(A, cmap='gray',
             extent=(0,1,0,1))
ax[0].set_title('A')
ax[0].set_axis_off()

ax[1].imshow(A_ruidosa,
             extent=(0,1,0,1),
             cmap='gray')
ax[1].set_title('A - ruidosa')

ax[2].imshow(Y_RBF, cmap='gray',
             extent=(0,1,0,1))
ax[2].set_title('A - RBF')


ax[3].imshow(Y_Matern,
             extent=(0,1,0,1),
             cmap='gray')
ax[3].set_title('A - Matern')


Z = m.feature.Z.read_value()
Z_1 = [x[0] for x in Z]
Z_2 = [x[1] for x in Z]
ax[2].plot(Z_1, Z_2, 'o', label = 'entradas inductoras estimadas')

Z = m1.feature.Z.read_value()
Z_1 = [x[0] for x in Z]
Z_2 = [x[1] for x in Z]
ax[3].plot(Z_1, Z_2, 'o', label = 'entradas inductoras estimadas')

plt.tight_layout()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

¿Qué pasaría si usaramos una escala más pequeña en el núcleo RBF?. Piensa la respuesta antes de ejecutar las celdas siguientes. Comenza Los valores estimados del kernel son los siguientes:

In [ ]:
m.kern.as_pandas_table()

Ejecutamos el modelo RBF con distintas escalas entre 0.15 y 0.01 y las pintamos.

In [ ]:
#Modelo con 100 inducing points
M = 100
Z = kmeans2(l, M, minit='points')[0]

sc = np.linspace(0.15, 0.01, 10)
Y_RBF_list = []

for sc_i in sc:
    var = 0.71
    var_lik = 0.08
    k = gpflow.kernels.RBF(2, lengthscales=sc_i, variance = var)
    m = gpflow.models.GPRFITC(l, A_ruidosa.reshape(-1,1), kern=k, Z=Z)

    #Predicción
    Y_pred, var = m.predict_y(l)
    Y_RBF = Y_pred.reshape(300,300)
    Y_RBF_list.append(Y_RBF)

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(14,14))
ax = axes.ravel()

ax[0].imshow(A, cmap='gray',
             extent=(0,1,0,1))
ax[0].set_title('A')
ax[0].set_axis_off()

ax[1].imshow(A_ruidosa,
             extent=(0,1,0,1),
             cmap='gray')
ax[1].set_title('A - ruidosa')

for i in range(2,12):
    ax[i].imshow(Y_RBF_list[i-2], cmap='gray',
                 extent=(0,1,0,1))
    ax[i].set_title('A - RBF, sc = {0:.2f}'.format(sc[i-2]))


plt.tight_layout()
plt.show()

¿Y si aumentamos la escala?